### Generation of masks with lines to determine volume calculation area

In [2]:
import pathlib
import os
import collections
from PIL import Image
import cv2
import numpy as np
import skimage.draw
import torch.utils.data
import loader
from functools import reduce
import operator
import math
import tqdm

def _defaultdict_of_lists():
    """Returns a defaultdict of lists.
    This is used to avoid issues with Windows (if this function is anonymous,
    the Echo dataset cannot be used in a dataloader).
    """

    return collections.defaultdict(list)

# Capture and Make Frames + Crop
def returnTracings(method="Method of Disks"):
    trace = collections.defaultdict(_defaultdict_of_lists)
    frames = collections.defaultdict(list)

    fnames = []

    root = "/Users/ishan/Documents/Stanford/EchoData"

    with open(os.path.join(root, "FileList.csv")) as f:
        header = f.readline().strip().split(",")
        filenameIndex = header.index("FileName")

        for line in f:
            lineSplit = line.strip().split(',')
            fileName = lineSplit[filenameIndex]

            if os.path.exists(os.path.join(root, "Videos", fileName)):
                fnames.append(fileName.split('.')[0])

        with open(os.path.join(root, "VolumeTracings.csv")) as f:
            header = f.readline().strip().split(",")
            assert header == ["FileName", "X1", "Y1", "X2", "Y2", "Frame"]

            for line in f:
                filename, x1, y1, x2, y2, frame = line.strip().split(',')
                x1 = float(x1)
                y1 = float(y1)
                x2 = float(x2)
                y2 = float(y2)
                frame = int(frame)

                if frame not in trace[filename]:
                    frames[filename].append(frame)
                trace[filename][frame].append((x1, y1, x2, y2))

        for filename in frames:
            for frame in frames[filename]:
                trace[filename][frame] = np.array(trace[filename][frame])

    return trace, fnames, frames, root

trace, fnames, frames, root = returnTracings()

In [3]:
root

'/Users/ishan/Documents/Stanford/EchoData'

In [7]:
for index in fnames:
    for frame_index in [0, -1]:
        video = os.path.join(root, "Videos", index + ".avi")
        video = loader.loadvideo(video).astype(np.float32)

        t = trace[index][frames[index][frame_index]]
        x1, y1, x2, y2 = t[:, 0], t[:, 1], t[:, 2], t[:, 3]
        x = np.concatenate((x1[1:], np.flip(x2[1:])))
        y = np.concatenate((y1[1:], np.flip(y2[1:])))

        r, c = skimage.draw.polygon(np.rint(y).astype(np.int), np.rint(x).astype(np.int), (video.shape[2], video.shape[3]))
        mask = np.zeros((video.shape[2], video.shape[3]), np.float32)
        mask[r, c] = 1

        rescaled = (255.0 / mask.max() * (mask - mask.min())).astype(np.uint8)

        im = Image.fromarray(rescaled)
        im.save("hello.png")

[[51.26041667 15.34895833 64.93229167 69.125     ]
 [50.03761083 17.16784126 53.36722189 16.32132997]
 [49.15737821 20.40762939 57.09054885 18.3907216 ]
 [48.5381733  23.58105455 59.99733911 20.66770731]
 [47.91896839 26.7544797  62.90412937 22.94469301]
 [47.9621045  29.75951307 65.81091963 25.22167871]
 [48.16791529 32.72318847 68.24704281 27.61832554]
 [48.37372608 35.68686387 70.38531105 30.0906982 ]
 [48.57953687 38.65053927 72.5235793  32.56307085]
 [49.01403934 41.55607271 74.15164369 35.16515635]
 [49.83099892 44.3643713  75.71299787 37.78420208]
 [50.64795851 47.17266989 77.27435205 40.4032478 ]
 [51.4649181  49.98096847 78.64469427 43.07085589]
 [52.54732826 52.72177963 79.48537463 45.87312377]
 [53.7042693  55.44364225 80.326055   48.67539165]
 [54.36563    58.29149988 80.609375   51.61936132]
 [51.15135239 62.12468928 80.609375   54.63536149]
 [49.33671909 65.6020369  80.4312144  57.69665674]
 [50.3482871  68.36085877 80.06545076 60.80564767]
 [57.51297555 69.55532799 79.69

[[50.53125    22.640625   61.46875    71.13020833]
 [50.54293236 23.53687283 51.65385953 23.28628775]
 [50.9013971  26.06518553 54.26366192 25.30677993]
 [51.23671767 28.59871872 56.90370306 27.32045134]
 [51.57203824 31.13225191 59.60385503 29.32056391]
 [51.90735881 33.6657851  61.75630627 31.444218  ]
 [52.24267938 36.19931829 63.38646261 33.68568297]
 [52.32768718 38.789313   65.01661896 35.92714794]
 [52.28801873 41.40743018 65.94012233 38.32800832]
 [52.24835029 44.02554736 66.71691579 40.76196115]
 [52.20868185 46.64366453 67.49370925 43.19591399]
 [52.12200806 49.27238442 68.20990123 45.64353634]
 [51.43073409 52.03748029 68.78993692 48.12187063]
 [50.73946013 54.80257615 69.36997261 50.60020492]
 [50.04818616 57.56767202 69.95000829 53.07853921]
 [49.35691219 60.33276789 70.22108934 55.62656252]
 [48.99891964 63.02268749 70.43805791 58.18679165]
 [48.87332445 65.66018665 70.65502648 60.74702077]
 [48.74772925 68.2976858  70.87199505 63.3072499 ]
 [50.16949051 70.58615718 70.89

[[47.43229167 23.734375   54.17708333 69.671875  ]
 [46.92044556 25.01399026 49.01251658 24.7068211 ]
 [46.33726549 27.50854241 51.02297322 26.82056151]
 [45.87280529 29.98566347 52.65045171 28.99053284]
 [45.609375   32.43326823 53.99776871 31.20163899]
 [45.609375   34.84219473 54.921308   33.47496648]
 [45.609375   37.25112124 55.84484729 35.74829396]
 [45.89883969 39.61754697 56.55405104 38.05309134]
 [46.30735213 41.96649348 57.21997613 40.36424313]
 [46.71586456 44.31543998 57.88590122 42.67539492]
 [47.10209365 46.66765825 58.55182631 44.98654671]
 [47.34997012 49.04019019 59.19234869 47.30142826]
 [47.59784659 51.41272214 59.71587235 49.63348819]
 [47.84572306 53.78525408 60.23939601 51.96554813]
 [48.09359953 56.15778602 60.76291967 54.29760807]
 [48.34147601 58.53031797 61.21037675 56.64083651]
 [48.21005796 60.95853998 61.51542674 59.00497393]
 [48.0751281  63.38727761 61.82047673 61.36911134]
 [47.96028877 65.81306545 61.99086919 63.75301991]
 [47.89492326 68.23158927 62.15

[[55.63541667 18.265625   57.09375    62.5625    ]
 [55.25787015 18.70685253 56.40653368 18.66903644]
 [54.50277713 21.04764091 57.9487677  20.93419266]
 [53.77613772 23.38749255 59.49100172 23.19934888]
 [53.35252399 25.71736804 61.03323574 25.4645051 ]
 [52.92891027 28.04724352 61.79062005 27.75549999]
 [52.50529654 30.37711901 62.2391518  30.05666287]
 [52.08168282 32.7069945  62.68768355 32.35782575]
 [51.65806909 35.03686998 63.1362153  34.65898863]
 [51.27738154 37.36533226 63.58474704 36.9601515 ]
 [51.55419946 39.67214827 64.03327879 39.26131438]
 [51.83101738 41.97896428 64.48181054 41.56247726]
 [52.1078353  44.28578029 64.93034229 43.86364014]
 [52.38465322 46.59259629 65.37887403 46.16480302]
 [52.66147114 48.8994123  65.99974191 48.46029228]
 [52.71385396 51.21361712 66.91407308 50.74612019]
 [51.32269869 53.57534582 67.82840424 53.03194811]
 [49.93154343 55.93707453 68.74273541 55.31777603]
 [48.54038816 58.29880324 69.65706658 57.60360395]
 [47.1492329  60.66053194 70.18

[[46.703125   18.99479167 59.828125   73.31770833]
 [45.96262001 20.64035831 51.58490062 19.28195494]
 [44.48161003 23.93149159 55.72869805 21.21407435]
 [43.43486319 27.11770224 58.99691487 23.35774344]
 [43.48052371 30.03997554 62.26513168 25.50141254]
 [43.52618423 32.96224883 65.53334849 27.64508163]
 [43.57184475 35.88452213 68.80156531 29.78875072]
 [43.69688786 38.78761574 70.87692853 32.22062605]
 [44.01428982 41.64423339 72.38558722 34.78942328]
 [44.33169178 44.50085103 73.89424591 37.35822051]
 [44.64909374 47.35746868 75.4029046  39.92701774]
 [44.9664957  50.21408633 76.71113864 42.54423971]
 [44.57867831 53.24109254 77.25253054 45.34673899]
 [43.88505784 56.34198407 77.79392245 48.14923826]
 [43.19143737 59.44287559 78.33531435 50.95173753]
 [42.4978169  62.54376711 78.87670626 53.7542368 ]
 [42.06908961 65.5806576  79.41809816 56.55673607]
 [42.52853359 68.40295637 79.0026491  59.5904184 ]
 [42.98797758 71.22525514 77.56361808 62.87140911]
 [44.9336658  73.68846134 76.12

...

[[51.26041667 17.53645833 56.72916667 68.76041667]
 [51.17056224 18.86181118 51.91973045 18.7818288 ]
 [50.99085338 21.51251686 53.238358   21.27256975]
 [50.81114452 24.16322254 54.55496385 23.76352653]
 [50.63143566 26.81392823 55.69956603 26.27284669]
 [50.46312498 29.46341702 56.84416821 28.78216685]
 [50.30917407 32.11137275 57.98877039 31.29148702]
 [50.15522315 34.75932847 59.13337257 33.80080718]
 [50.00127224 37.4072842  59.96960999 36.34304885]
 [49.84732132 40.05523992 60.76009737 38.89017486]
 [49.69337041 42.70319565 61.55058475 41.43730088]
 [49.35564266 45.37077167 62.34107214 43.98442689]
 [48.84967174 48.0563096  63.13155952 46.5315529 ]
 [48.34370083 50.74184754 63.9220469  49.07867892]
 [47.83772991 53.42738547 64.71253429 51.62580493]
 [47.331759   56.1129234  65.50302167 54.17293094]
 [46.73170973 58.80850528 66.29350906 56.72005696]
 [46.11352859 61.50602295 67.08399644 59.26718297]
 [45.49534746 64.20354062 67.54447961 61.84954075]
 [45.45512971 66.83935402 67.66

[[49.61979167 18.44791667 63.109375   70.03645833]
 [50.71354167 21.72916667 58.55208333 67.30208333]
 [52.90104167 24.64583333 61.10416667 65.84375   ]
 [50.4082829  22.52197586 53.1009301  22.05884054]
 [45.79166667 20.90188135 51.94409707 19.29311863]
 [49.06859871 25.20134425 56.65792501 23.89598013]
 [45.53588018 23.87470718 55.33702494 21.31186368]
 [52.15212966 25.51550919 54.80651548 24.98698104]
 [47.76897656 27.87382197 57.407118   26.21606164]
 [51.20889298 27.87360089 58.034824   26.51445533]
 [47.21902311 30.41735667 58.15631098 28.53614316]
 [45.05062778 26.9075347  57.61254912 23.62279202]
 [50.2656563  30.23169258 59.05121236 28.4823562 ]
 [46.66906966 32.96089137 58.90550397 30.85622467]
 [44.56537538 29.94036222 59.45214172 26.04770954]
 [49.65091659 32.52437559 60.06760072 30.45025707]
 [46.11911621 35.50442607 59.65469696 33.17630618]
 [44.26514739 32.9248088  61.18207071 28.50130235]
 [49.35365412 34.75384411 61.08398908 32.41815794]
 [45.56916276 38.04796078 60.40

[[45.97395833 18.08333333 58.734375   67.11979167]
 [44.64225352 19.74796435 47.55886564 18.98899465]
 [44.01081511 22.54846064 50.72868026 20.80031729]
 [44.34139114 25.09861859 52.24494337 23.04193214]
 [44.67196718 27.64877655 53.55633091 25.33686034]
 [45.00254321 30.1989345  54.86771845 27.63178853]
 [45.63641688 32.67016742 56.17910599 29.92671673]
 [46.33748738 35.12391418 57.02498321 32.34278144]
 [47.03855789 37.57766094 57.86972685 34.75914113]
 [47.73962839 40.03140769 58.71447048 37.17550083]
 [48.38683982 42.49916982 59.55921411 39.59186052]
 [48.69843697 45.05426652 60.40395775 42.00822022]
 [49.01003413 47.60936321 61.24870138 44.42457992]
 [49.32163129 50.16445991 62.09344501 46.84093961]
 [49.63322845 52.71955661 62.93818865 49.25729931]
 [49.94482561 55.2746533  63.78293228 51.673659  ]
 [50.14347297 57.85914214 64.54782283 54.11079831]
 [50.06295374 60.51627656 65.28140849 56.55608387]
 [49.98243452 63.17341098 66.01499416 59.00136943]
 [49.90191529 65.83054539 66.74

[[58.36979167 14.984375   65.84375    70.03645833]
 [54.41557129 16.94508826 60.60943943 16.10419888]
 [53.23033631 19.95376169 65.08873494 18.34384664]
 [52.17739953 22.94447409 69.14676501 20.64068607]
 [52.22904956 25.78522604 70.60845992 23.29000808]
 [52.2806996  28.62597799 72.07015482 25.9393301 ]
 [52.33234964 31.46672994 73.53184973 28.58865211]
 [52.65492308 34.27070089 74.90905623 31.24944441]
 [53.17562424 37.04777371 75.80295992 33.97585066]
 [53.69632539 39.82484654 76.6968636  36.70225691]
 [54.11671082 42.61553838 77.59076729 39.42866316]
 [54.40237006 45.42452087 78.45635247 42.15891398]
 [54.6880293  48.23350335 78.94714079 44.94004782]
 [54.97368853 51.04248584 79.43792912 47.72118165]
 [54.72924718 53.92343562 79.92871744 50.50231549]
 [54.12835653 56.85277753 80.41950576 53.28344932]
 [53.41275897 59.79769223 80.3832417  56.13613663]
 [52.16180411 62.81528789 80.01442634 59.03397163]
 [51.74693448 65.71937529 79.64561097 61.93180663]
 [51.33206485 68.62346269 79.06

KeyboardInterrupt: 

In [8]:
import pandas as pd
from ast import literal_eval
import os
import cv2
from tqdm import tqdm
from algorithms import funcs as funcs
import loader
from PIL import Image 
import skimage.draw
import numpy as np 

root = "/Users/ishan/Documents/Stanford/EchoData"
PATH_TO_SEGMENTED_FRAMES_PARENT_DIR = os.path.join(root, "Masks_From_VolumeTracing")

print("\nIterating through each video and making frame if selected")
for i in tqdm(os.listdir(PATH_TO_SEGMENTED_FRAMES_PARENT_DIR)): # iterates through each row of data frame
    videoName = i.split("_")[0]
    frame = i.split("_")[1].split(".")[0]

    videoPath = os.path.join(root, "Videos", videoName)
    
    video = loader.loadvideo(videoPath).astype(np.float32)
    
    INPUT_FRAME_PATH = os.path.join(PATH_TO_SEGMENTED_FRAMES_PARENT_DIR, i) # (data directory)/frames/(file name)

    volumes, x1, y1, x2, y2 = funcs.calculateVolumeErosionAndDilation(INPUT_FRAME_PATH, 20, iterations=10, method="Method of Disks")
    
    for iteration in x1:
        mask = loader.READ_AND_CROP_FRAME(videoPath, frame, False)
        x = np.concatenate((x1[iteration][1:], np.flip(x2[iteration][1:])))
        y = np.concatenate((y1[iteration][1:], np.flip(y2[iteration][1:])))

        r, c = skimage.draw.polygon(np.rint(y).astype(np.int), np.rint(x).astype(np.int), (video.shape[2], video.shape[3]))
        #mask = np.zeros((video.shape[2], video.shape[3]), np.float32)
        
        mask[r, c] = -1

        rescaled = (255.0 / mask.max() * (mask - mask.min())).astype(np.uint8)
        
        if videoName == "0X5A0750FDDF8BB7FD.avi":
            im = Image.fromarray(rescaled)
            im.save(str(iteration) + ".png")        


  0%|          | 1/19943 [00:00<37:28,  8.87it/s]


Iterating through each video and making frame if selected


  0%|          | 39/19943 [00:04<41:24,  8.01it/s]


IndexError: list index out of range